In [1]:
import joblib
from sklearn.ensemble import RandomForestClassifier
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client import Point, WritePrecision
import time


ModuleNotFoundError: No module named 'influxdb_client'

In [ ]:
# InfluxDB configuration
url = 'http://localhost:8086'
token = 'your influxdb token'
org = 'Organisation name saved in influxdb'
bucket = 'Your bucket name'

In [ ]:
def fetch_most_recent_weather(city='delhi'):
    try:
        
        client = InfluxDBClient(url=url, token=token, org=org)
        
        
        query = f'from(bucket: "{bucket}") \
                  |> range(start: -1h) \
                  |> filter(fn: (r) => r["_measurement"] == "weather" and r["_field"] =~ /^(humidity|wind_speed|pressure)$/) \
                  |> filter(fn: (r) => r["city"] == "{city}") \
                  |> last()'
        
        
        tables = client.query_api().query(query=query)
        
        
        humidity = None
        wind_speed = None
        pressure = None
        timestamp = None
        
       
        for table in tables:
            for record in table.records:
               
                humidity = record.get_value()
                wind_speed = record.get_value()
                pressure = record.get_value()
                timestamp = record.get_time()
                
            
                print(f'City: {city}, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s, Pressure: {pressure} hPa, Timestamp: {timestamp}')
                
                
                return city, humidity, wind_speed, pressure
    
    except Exception as e:
        print(f'Error fetching data from InfluxDB: {e}')
    

    return None, None, None, None


city, humidity, wind_speed, pressure = fetch_most_recent_weather()

In [ ]:

def load_model(file_path):
    model = joblib.load(file_path)
    return model

model_path = r'saved random forest path'
model = load_model(model_path)

In [ ]:
def predict_weather(month, humidity, wind_speed, pressure, model):
    features = [[month, humidity, wind_speed, pressure]]
    prediction = model.predict(features)
    return prediction

In [ ]:
from datetime import datetime
def findmonth():
    current_datetime = datetime.now()
    month = current_datetime.month
    return month


In [ ]:
month = findmonth()
prediction = float(predict_weather(month, humidity, wind_speed, pressure, model))
print(f'Mean Temperature: {prediction}')


In [ ]:
def store_predicted_value(city, predicted_Temp):
    try:
       
        client = InfluxDBClient(url=url, token=token, org=org)

        
        point = Point("predicted_Temp") \
            .tag("city", city) \
            .field("predicted_value", predicted_Temp)

        
        write_api = client.write_api(write_options=SYNCHRONOUS)
        write_api.write(bucket=bucket, record=point)

        print(f'Predicted value {predicted_Temp} for {city} stored successfully in InfluxDB.')

    except Exception as e:
        print(f'Error storing predicted value in InfluxDB: {e}')


In [ ]:
while True:
    month = findmonth()
    city, humidity, wind_speed, pressure = fetch_most_recent_weather()
    prediction = float(predict_weather(month, humidity, wind_speed, pressure, model))
    store_predicted_value(city, prediction)
    time.sleep(10)

KeyboardInterrupt: 